![alt text](../img/header.jpg)

# Exercise x: MODFLOW 6
The purpose of this exercise is to use MODFLOW 6 to simulate the formation of a freshwater lens in an axisymmetric island.  This problem requires use of the Buoyancy Package and the Groundwater Transport (GWT) Model, both of which are included in the mf6beta program, which is not an official USGS release.  The DISU package is used for this problem, which allows an axisymmetric grid to be constructed.  This example is described by Bedekar et al. (2019) in the context of a MODFLOW-USG simulation (https://onlinelibrary.wiley.com/doi/abs/10.1111/gwat.12861).

## Part I. Setup Notebook

In [ ]:
%matplotlib inline
import sys
import os
import shutil
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
import flopy

import config

print(sys.version)
print('python executable: {}'.format(sys.executable))
print('numpy version: {}'.format(np.__version__))
print('matplotlib version: {}'.format(mpl.__version__))
print('flopy version: {}'.format(flopy.__version__))

## Part II. Create the Axisymmetric Model Grid

In [ ]:
# model info
model_ws = './ex0x-completed-axisym'

# grid properties
nlay = 70
nrow = 1
ncol = 101
nodes = nlay * nrow * ncol
delr = 5.
delc = 1.
delz = 0.5
radius_edge = np.linspace(delr, ncol * delr, ncol)

# hydraulic properties
hk = 10.
ss = 1.e-4
rech = 1.74e-003

In [ ]:
# unstructured grid arrays
nja = (ncol - 2) * (nlay - 2) * 5  # interior
nja += 2 * (nlay - 2) * 4 + 2 * (ncol - 2) * 4  # edges (excluding corners)
nja += 4 * 3  # corners

print('Number of nodes: {}'.format(nodes))
print('Number of connections, including diagonal: {}'.format(nja))

iac = np.zeros(nodes, dtype=np.int)
ia = np.zeros(nodes + 1, dtype=np.int)
area = np.zeros(nodes, dtype=np.float)
top = np.zeros(nodes, dtype=np.float)
bot = np.zeros(nodes, dtype=np.float)
ja = np.zeros(nja, dtype=np.int)
cl12 = np.zeros(nja, dtype=np.float)
ihc = np.zeros(nja, dtype=np.int)
hwva = np.zeros(nja, dtype=np.float)

In [ ]:
ipos = 0
for k in range(nlay):
    for j in range(ncol):
        
        # diagonal
        n = (k * ncol) + j
        ja[ipos] = n
        a = np.pi * radius_edge[j] ** 2
        if j > 0:
            a -= np.pi * radius_edge[j - 1] ** 2
        area[n] = a
        top[n] = 0. - k * delz
        bot[n] = top[n] - delz
        iac[n] += 1
        ipos += 1
        
        # up
        if k > 0:
            m = n - ncol
            ja[ipos] = m
            iac[n] += 1
            cl12[ipos] = .5 * delz
            hwva[ipos] = a
            ipos += 1
            
        # left
        if j > 0:
            m = n - 1
            ja[ipos] = m
            iac[n] += 1
            cl12[ipos] = .5 * delr
            hwva[ipos] = 2. * np.pi * radius_edge[j - 1]
            ihc[ipos] = 1
            ipos += 1

        # right
        if j < ncol - 1:
            m = n + 1
            ja[ipos] = m
            iac[n] += 1
            cl12[ipos] = .5 * delr
            hwva[ipos] = 2. * np.pi * radius_edge[j]
            ihc[ipos] = 1
            ipos += 1

        # down
        if k < nlay - 1:
            m = n + ncol
            ja[ipos] = m
            iac[n] += 1
            cl12[ipos] = .5 * delz
            hwva[ipos] = a
            ipos += 1
            
        ia[n + 1] = ia[n] + iac[n]

In [ ]:
vertices = []
ivert = 0
for x in np.linspace(0, ncol * delr, ncol + 1):
    vertices.append((ivert, x, delc))
    ivert += 1
    vertices.append((ivert, x, 0.))
    ivert+= 1
nvert = len(vertices)

cell2d = []
n = 0
for k in range(nlay):
    for j in range(ncol):
        xc = radius_edge[j] - .5 * delr
        yc = .5 * delc
        iv1 = j * 2
        cell2d.append((n, xc, yc, 4, iv1, iv1 + 1, iv1 + 3, iv1 + 2))
        n += 1

## Part III. Fetter Solution

In [ ]:
def interface(r, R, w, k, G):
    return np.sqrt(w * (R ** 2 - r ** 2) / 2. / k / (1. + G))

In [ ]:
fetter_h = interface(radius_edge, radius_edge[-1], rech, hk, 40.)
fetter_z = -40. * fetter_h
plt.plot(radius_edge, fetter_h)
plt.plot(radius_edge, fetter_z)

## Part IV. Create, Run, and Post-Process MODFLOW 6 Model

In [ ]:
# create simulation
sim = flopy.mf6.MFSimulation(#sim_name=model_name, 
                             version='mf6', 
                             exe_name=config.mf6betaexe, 
                             sim_ws=model_ws)

# create tdis package
nper = 1
perlen = 10000.
nstp = 10000
tmult = 1.0
tdis = flopy.mf6.ModflowTdis(sim, time_units='DAYS',
                             nper=nper, perioddata=[(perlen, nstp, tmult)])

# create gwf model
gwfname = 'flow'
gwf = flopy.mf6.ModflowGwf(sim, modelname=gwfname,
                           save_flows=True)

# create iterative model solution and register the gwf model with it
hclose = 1.e-3
rclose = 1.e-3
outer_maximum = 250
inner_maximum = 600
relaxation_factor = 0.0
ims = flopy.mf6.ModflowIms(sim, filename=gwfname+'.ims', 
                           print_option='SUMMARY',
                           complexity='complex',
                           outer_hclose=hclose,
                           outer_maximum=outer_maximum,
                           inner_maximum=inner_maximum,
                           inner_hclose=hclose, 
                           rcloserecord=rclose,
                           relaxation_factor=relaxation_factor)
sim.register_ims_package(ims, [gwf.name])


# dis
dis = flopy.mf6.ModflowGwfdisu(gwf, nodes=nodes, nja=nja,
                               top=top, bot=bot, area=area,
                               iac=iac, ja=ja+1, cl12=cl12,
                               ihc=ihc, hwva=hwva, 
                               nvert=nvert, vertices=vertices, cell2d=cell2d)

# initial conditions
ic = flopy.mf6.ModflowGwfic(gwf, strt=0.)

# node property flow
npf = flopy.mf6.ModflowGwfnpf(gwf, icelltype=0, k=hk)

# storage
sto = flopy.mf6.ModflowGwfsto(gwf, iconvert=0, ss=ss)

# constant head
chdspd = {0: [((n,), 0.) for n in range(ncol-1, ncol*nlay + ncol-1, ncol)]}
chd = flopy.mf6.ModflowGwfchd(gwf, stress_period_data=chdspd)

# recharge
rchspd = {0: [((n,), rech) for n in range(ncol)]}
rch = flopy.mf6.ModflowGwfrch(gwf, stress_period_data=rchspd)

# buoyancy
buy = flopy.mf6.ModflowGwfbuy(gwf, drhodc=0.7143, hhformulation=True)

# output control
oc = flopy.mf6.ModflowGwfoc(gwf,
                            budget_filerecord='{}.cbc'.format(gwfname),
                            head_filerecord='{}.hds'.format(gwfname),
                            headprintrecord=[
                                ('COLUMNS', 10, 'WIDTH', 15,
                                 'DIGITS', 6, 'GENERAL')],
                            saverecord=[('HEAD', 'LAST'),
                                        ('BUDGET', 'LAST')],
                            printrecord=[('HEAD', 'LAST'),
                                         ('BUDGET', 'ALL')])


# create gwt model
gwtname = 'transport'
gwt = flopy.mf6.MFModel(sim, model_type='gwt6', modelname=gwtname)
gwt.name_file.save_flows = True

# create iterative model solution and register the gwt model with it
imsgwt = flopy.mf6.ModflowIms(sim, filename=gwtname+'.ims', 
                              print_option='SUMMARY', 
                              outer_hclose=hclose,
                              outer_maximum=outer_maximum,
                              under_relaxation='NONE',
                              inner_maximum=inner_maximum,
                              inner_hclose=hclose, rcloserecord=rclose,
                              linear_acceleration='BICGSTAB',
                              scaling_method='NONE',
                              reordering_method='NONE',
                              relaxation_factor=0.97)
sim.register_ims_package(imsgwt, [gwt.name])

disu = flopy.mf6.ModflowGwtdisu(gwt, nodes=nodes, nja=nja,
                                top=top, bot=bot, area=area,
                                iac=iac, ja=ja+1, cl12=cl12,
                                ihc=ihc, hwva=hwva)

# initial conditions
ic = flopy.mf6.ModflowGwtic(gwt, strt=35.)

# advection
adv = flopy.mf6.ModflowGwtadv(gwt, scheme='tvd')

# storage
sto = flopy.mf6.ModflowGwtsto(gwt, porosity=0.1)

# constant concentration
cncspd = {0: [((n,), 35.) for n in range(ncol-1, ncol*nlay + ncol-1, ncol)]}
cnc = flopy.mf6.ModflowGwtcnc(gwt, stress_period_data=cncspd)


# sources
sourcerecarray = [()]
ssm = flopy.mf6.ModflowGwtssm(gwt, sources=sourcerecarray)

# output control
oc = flopy.mf6.ModflowGwtoc(gwt,
                            budget_filerecord='{}.cbc'.format(gwtname),
                            concentration_filerecord='{}.ucn'.format(gwtname),
                            concentrationprintrecord=[
                                ('COLUMNS', 10, 'WIDTH', 15,
                                 'DIGITS', 6, 'GENERAL')],
                            saverecord=[('CONCENTRATION', 'LAST'),
                                        ('BUDGET', 'LAST')],
                            printrecord=[('CONCENTRATION', 'LAST'),
                                         ('BUDGET', 'LAST')])

gwfgwt = flopy.mf6.ModflowGwfgwt(sim, exgtype='GWF6-GWT6',
                                 exgmnamea=gwfname, exgmnameb=gwtname)


sim.write_simulation()
sim.run_simulation()

In [ ]:
fname = os.path.join(model_ws, gwtname + '.ucn')
os.path.exists(fname)
cnobj = flopy.utils.HeadFile(fname, text='CONCENTRATION')
times = cnobj.get_times()
print('len times: {}'.format(len(times)))
conc = cnobj.get_data(totim=times[-1])
conc = conc.reshape((nlay, ncol))

In [ ]:
fig, ax = plt.subplots(figsize=(10,5))
cs = ax.contour(np.flipud(conc), levels=[f * 35. for f in [.1, .5, .9]], 
                extent=[2.5, 502.5, -35, 0])
cs.collections[0].set_label('MF 6')
ax.plot(radius_edge, fetter_z, 'k--')
ax.set_xlabel('Distance from Island Center (m)')
ax.set_ylabel('Elevation (m)');

In [ ]:
fig, ax = plt.subplots(figsize=(10,5))
cs = ax.contourf(np.flipud(conc), levels=[f * 35. for f in [0, .1, .5, .9, 1.0]], 
                extent=[2.5, 502.5, -35, 0], cmap='jet')
ax.plot(radius_edge, fetter_z, 'k--')
ax.set_xlabel('Distance from Island Center (m)')
ax.set_ylabel('Elevation (m)')
plt.colorbar(cs, shrink=0.5);